
## Elaboração de modelo de classificação para predizer a probabilidade de cancelamento de uma reserva por um cliente

Esse caderno irá introduzir um modelo classificatório para prever a probabilidade de um determinado cliente cancelar uma reserva em um hotel.

**Conjunto de Dados**: https://www.sciencedirect.com/science/article/pii/S2352340918315191

O caderno será construído utilizando um framework padrão de Ciência de Dados (Crisp-DM).

O caderno será dividido, então, nas seguintes etapas:

* **Análise exploratória de dados (AED)** - o processo de percorrer um conjunto de dados e descobrir mais informações sobre ele.
* **Treinamento do modelo** - criar modelo(s) para aprender a prever uma variável alvo com base em outras variáveis.
* **Avaliação do modelo** - avaliar as previsões de um modelo usando métricas de avaliação específicas do problema.
* **Comparação dos modelos** - Nessa etapa, vamos comparar os três modelos escolhidos.
* **Ajuste do modelo** - como podemos melhorar o modelo?
* **Importância das variáveis** - já que estamos prevendo a chance de um cliente cancelar uma reserva, existem atributos mais importes para se chegar a essa predição?
* **Validação cruzada** - se construirmos um bom modelo, podemos ter certeza de que funcionará em dados não vistos?
* **Relatando o que encontramos** - se precisássemos apresentar nosso trabalho, o que mostraríamos a alguém?

Vamos utilizar três modelos nessa análise: Logistic Regression, K-Nearest Neighbors e RandomForest.

* **Métrica de avalição escolhida**: prever com grau de 80% de acurácia se um cliente irá cancelar uma reserva.

**Dicionário do Conjunto de Dados: (Nome da entrada, tipo de dado, descrição)**

* 1. ADR - Numérico - Média de transações diárias ("The average daily rate (ADR) is a performance indicator used in the hospitality sector to measure the strength of revenues generated. It is measured as the total revenues generated by all the occupied rooms in a hotel or lodge divided by the total number of occupied rooms over a given time period.) 
* 2. Adults - Inteiro - número de adultos na reserva
* 3. Agent - Categórico - Agente que efetuou a reserva
* 4. ArrivalDateDayOfMonth - Inteiro - Dia do mês da data de chegada
* 5. ArrivalDateMonth - Categorical - Mês da chegada
* 6. ArrivalDateWeekNumber - Integer - Número da semana da chegada
* 7. ArrivalDateYear - Integer -Ano da Chegada
* 8. AssignedRoomType - Categórico - Código do tipo de quarto reservado
* 9. Babies - Inteiro - Número de Bebês
* 10.BookingChanges - Inteiro - Número de modificações feito na reserva até a data de checkin
* 11. Children - Inteiro - Número de crianças na reserva
* 12. Company - Categórico - ID da empresa que fez a reserva
* 13. Country - Categórico - país de origem
* 14. CustomerType - Categórico - Tipo de clinte que fez a reserva (grupo, agência de viagens, sozinho, etc...)
* 15. DaysInWaitingList - Inteiro - Tempo em dias que o cliente ficou em lista de espera
* 16. DepositType - Categórico - Se fez um depósito para reservar (restituível, sem depósito e sem restituição)
* 17. DistributionChannel - Categórico - Canal em que feita a reserva
* 18. IsCanceled - Categórico - Valor que diz que se a reserva foi ou não cancelada
* 19. IsRepeatedGuest - Categórico - se é ou não um hóspede repetitivo
* 20. LeadTime - Inteiro - tempo entre a reserva e a data de chegada
* 21. MarketSegment - Categórico - segmento de mercado
* 22. Meal - Categórico - tipo de refeição agendada
* 23. PreviousBookingsNotCanceled - Inteiro - número de reservas prévias NÂO canceladas 
* 24. PreviousCancellations - Inteiro - Núero de reservas prévias canceladas
* 25. RequiredCardParkingSpaces - Inteiro - Número de reservas de garagem solicitadas
* 26. ReservationStatus - Categorical ( cancelado, no-show, checked-out)
* 27. ReservationStatusDate - Data - data do último status da reserva
* 28. ReservedRoomType - Categorical - código do quarto reservado
* 29. StaysInWeekendNights - Inteiro - número de dias de final de semana da reserva
* 30. StaysInWeekNights - Inteiro - número de dias da semana da reserva
* 31. TotalOfSpecialRequests - Inteiro - número de solicitações especiais feitas pelo cliente


## Ferramentas utilizadas
* pandas for data analysis.
* NumPy for numerical operations.
* Matplotlib/seaborn for plotting or data visualization.
* Scikit-Learn for machine learning modelling and evaluation.


## 1. Análise Exploratória dos Dados

In [1]:
import numpy as np # np is short for numpy
import pandas as pd # pandas is so commonly used, it's shortened to pd
import matplotlib.pyplot as plt
import seaborn as sns # seaborn gets shortened to sns


%matplotlib inline 

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

## Avaliadores do Modelo

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import RocCurveDisplay # new in Scikit-Learn 1.2+


import time
print(f"Last updated: {time.asctime()}")

Last updated: Tue Nov  7 10:39:07 2023


In [2]:
df = pd.read_csv("H2.csv") 
df.shape 

(79330, 31)

In [ ]:
## Vamos retirar a coluna "ReservationStatus" porque ela informa se a reserva foi 
## ou não cancelada, além da coluna "IsCancelled", o que iria dar a resposta ao modelo.

## também vamos retirar a coluna "ADR" porque ela pode influenciar o resultado.

df = df.drop('ReservationStatus', axis=1)
df = df.drop('ADR', axis=1)

In [ ]:
df.head()

In [ ]:
df["ArrivalDateYear"].unique()

In [ ]:
df.IsCanceled.value_counts()

In [ ]:
df.IsCanceled.value_counts(normalize=True)

Podemos perceber, portanto, que a coluna IsCanceled, valor alvo para a predição do modelo, é relativamente balanceada, com cerca de 41% das reservas sendo canceladas, valor que aparentemente é alto.

In [ ]:
df.describe()

In [ ]:
df.info()

Vamos criar uma coluna 'ID' com um identificador único para cada linha.

In [ ]:
df['ID'] = range(1, len(df) + 1)

In [ ]:
df.head()

A seguir, a distribuição dos dados em função de algumas colunas importantes do conjunto de dados. Podemos ver que a grande maioria possui 2 adultos.

In [ ]:
df.groupby('Adults')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.2f%%',subplots=True,title='Adults',figsize=(5,5))

In [ ]:
df.groupby('Children')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.2f%%',subplots=True,title='Children',figsize=(5,5))

In [ ]:
df.groupby('StaysInWeekendNights')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='bar',figsize=(5,5))

In [ ]:
df.groupby('StaysInWeekNights')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='barh',figsize=(5,5))

In [ ]:
df.groupby('Meal')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.2f%%',subplots=True,title='Meal',figsize=(5,5))

In [ ]:
df.groupby('RequiredCarParkingSpaces')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.2f%%',subplots=True,title='Required car parking space',figsize=(5,5))

In [ ]:
df.groupby('ReservedRoomType')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='barh',figsize=(5,5))

In [ ]:
df.groupby('ArrivalDateYear')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.2f%%',subplots=True,title='Years',figsize=(5,5))

In [ ]:
df.groupby('ArrivalDateMonth')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.2f%%',subplots=True,title='Months',figsize=(9,9))

In [ ]:
df.groupby('ArrivalDateDayOfMonth')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.2f%%',subplots=True,title='Date',figsize=(9,9))

In [ ]:
df.groupby('IsRepeatedGuest')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.2f%%',subplots=True,title='Repeated guest',figsize=(5,5))

In [ ]:
df.groupby('PreviousCancellations')['ID'].agg(['count']).sort_values(
    by='count',ascending=False).plot(
    kind='pie',autopct='%1.3f%%',subplots=True,title='Cancellations',figsize=(5,5))

## Pré-Processamento

Vamos converter os valores categóricos em inteiros e lidar com os valores vazios na tabela.


In [ ]:
df_tmp = df.copy()

In [ ]:
df_tmp.head().T

In [ ]:
df_tmp.isna().sum()

Percebe-se que a coluna "Country" possui 24 valores faltantes e é uma coluna problemática. É problemática porque a maior parte das informações é feita com base nos dados da reserva, que muitas vezes são preenchidos em função do local em que é feita a reserva ou mesmo em virtude de informações de cartões de crédito, que não condizem muitas vezes com o país de origem do reservante. Além disso, a informação pode acrescentar um viés preconceituoso em relação a pessoas de determinados países. É possível que um país esteja passando por um momento turbulento em virtude de determinado acontecimento e, em função disso, naquele perído o número de cancelamentos aumente, influenciando desnecessariamente o modelo. Difícil perceber uma razão para o país de origem influenciar justificamente a previsão do modelo.

Assim, achamos melhor retirar da tabela a coluna de país de origem.

In [ ]:
df_tmp = df_tmp.drop('Country', axis=1)

In [ ]:
df_tmp.isna().sum()

In [ ]:
# These columns contain strings
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [ ]:
df_tmp.info()

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [ ]:
df_tmp.info()

In [ ]:
df_tmp.head()


In [ ]:
df_tmp.CustomerType.cat.categories

In [ ]:
df_tmp.CustomerType.cat.codes.unique()

In [ ]:
df_tmp.to_csv("H2_tmp.csv",
              index=False)

Como a única coluna com valores faltantes é a coluna de crianças, uma saída é preencher os 4 valores faltantes com a mediana da coluna, arredondada para um inteiro.

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df_tmp[label] = content.fillna(int(content.median()))

In [ ]:
df_tmp.isna().sum()

Pronto, os dados em string foram transformados para categóricos e os valores vazios foram substituídos pela mediana da coluna. Agora será necessário transformar os dados categóricos em numéricos para usar os modelos de classificação, que não operam com dados categóricos.

In [ ]:
label_encoder = LabelEncoder()

for column in df_tmp.columns:
    if df_tmp[column].dtype == 'category':
        df_tmp[column] = label_encoder.fit_transform(df_tmp[column])


In [ ]:
df_tmp.info()

In [ ]:
df_val = df_tmp[df_tmp.ArrivalDateYear == 2015]
df_train = df_tmp[df_tmp.ArrivalDateYear != 2015]


len(df_val), len(df_train)


In [ ]:
X = df_train.drop("IsCanceled", axis=1)
y = df_train.IsCanceled.values

In [ ]:
##df_emb = df_tmp.sample(frac=1, random_state=42)
##df_emb_train_and_test = df_tmp[:74330]
##df_emb_val=df_tmp[74331:]


##X_train_and_test = df_emb_train_and_test.drop("IsCanceled", axis=1)
##X_val = df_emb_val.drop("IsCanceled", axis=1)

# Target variable
##y_train_and_test = df_emb_train_and_test.IsCanceled.values
##y_val = df_emb_val.IsCanceled.values



In [ ]:
np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X, # variável independente
                                                    y, # variável dependente
                                                    test_size = 0.2)

Vamos colocar os três modelos em um dicionário e iterar o dicionário para testar os três modelos de uma só vez:

In [ ]:
models = {"SVC": LinearSVC(),
          "Logistic Regression": LogisticRegression(), 
          "Random Forest": RandomForestClassifier()}

def fit_and_score(models, X_train, X_test, y_train, y_test):
   
    np.random.seed(42)
    model_scores = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_test,
                             y_train=y_train,
                             y_test=y_test)
model_scores

In [ ]:
X_val = df_val.drop("IsCanceled", axis=1)
y_val = df_val.IsCanceled.values


np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X_val, # variável independente
                                                    y_val, # variável dependente
                                                    test_size = 0.2)

models['SVC'].score(X_val, y_val)


In [ ]:
X_val = df_val.drop("IsCanceled", axis=1)
y_val = df_val.IsCanceled.values


np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X_val, # variável independente
                                                    y_val, # variável dependente
                                                    test_size = 0.2)

models['Logistic Regression'].score(X_val, y_val)

In [ ]:
X_val = df_val.drop("IsCanceled", axis=1)
y_val = df_val.IsCanceled.values


np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X_val, # variável independente
                                                    y_val, # variável dependente
                                                    test_size = 0.2)

models['Random Forest'].score(X_val, y_val)

Vamos tentar aprimorar os parâmetros do modelo RandomForest, que se saiu melhor. 

In [ ]:
n_estimators = 210
min_samples_split = 4
min_samples_leaf = 19
max_depth = 3

randomForest = RandomForestClassifier(
    n_estimators=n_estimators,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    max_depth=max_depth
)



In [ ]:
X_val = df_val.drop("IsCanceled", axis=1)
y_val = df_val.IsCanceled.values


np.random.seed(42)


randomForest.fit(X_train, y_train)
randomForest.score(X_test, y_test)

In [ ]:
y_preds = randomForest.predict(X_test)

In [ ]:
y_preds

In [ ]:
RocCurveDisplay.from_estimator(estimator=randomForest, 
                               X=X_test, 
                               y=y_test); 

In [ ]:
print(confusion_matrix(y_test, y_preds))

In [ ]:
def plot_conf_mat(y_test, y_preds):
   
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds),
                     annot=True, 
                     cbar=False, 
                    fmt='d')
    plt.xlabel("Classificação verdadeira")
    plt.ylabel("Classificação realizada pelo modelo")
    
plot_conf_mat(y_test, y_preds)

In [ ]:
# Show classification report
print(classification_report(y_test, y_preds))

In [ ]:
cv_acc = cross_val_score(randomForest,
                         X,
                         y,
                         cv=5, # 5-fold cross-validation
                         scoring="accuracy") # accuracy as scoring
cv_acc

In [ ]:
cv_acc = np.mean(cv_acc)
cv_acc

In [ ]:
cv_precision = np.mean(cross_val_score(randomForest,
                                       X,
                                       y,
                                       cv=5, # 5-fold cross-validation
                                       scoring="precision")) # precision as scoring
cv_precision

In [ ]:
cv_recall = np.mean(cross_val_score(randomForest,
                                    X,
                                    y,
                                    cv=5, # 5-fold cross-validation
                                    scoring="recall")) # recall as scoring
cv_recall

In [ ]:
cv_f1 = np.mean(cross_val_score(randomForest,
                                X,
                                y,
                                cv=5, # 5-fold cross-validation
                                scoring="f1")) # f1 as scoring
cv_f1

In [ ]:
# Visualizing cross-validated metrics
cv_metrics = pd.DataFrame({"Accuracy": cv_acc,
                            "Precision": cv_precision,
                            "Recall": cv_recall,
                            "F1": cv_f1},
                          index=[0])
cv_metrics.T.plot.bar(title="Medidas de Validação Cruzada", legend=False);

In [ ]:
randomForest.coef_